<a href="https://colab.research.google.com/github/varti-pant/laptop_price_ml/blob/main/laptop_price_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install beautifulsoup4 openpyxl

In [19]:
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files

# Upload the HTML file
uploaded = files.upload()

# Assuming the file is named 'amazon.htm'
file_name = 'amazon.htm'

# Read the HTML file
with open(file_name, 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Initialize default values
company = ""
ram = ""
resolution = ""
screen_size = ""
memory = ""
weight = ""
opsys = ""
gpu = ""
cpu = ""

# Find the specific table
table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})

if table:
    # Iterate through the rows of the table
    for row in table.find_all('tr'):
        header = row.find('th', {'class': 'a-color-secondary a-size-base prodDetSectionEntry'})
        value = row.find('td', {'class': 'a-size-base prodDetAttrValue'})
        if header and value:
            header_text = header.get_text(strip=True)
            value_text = value.get_text(strip=True)
            if header_text == "Brand":
                company = value_text
            elif header_text == "RAM Size":
                ram = value_text
            elif header_text == "Resolution":
                resolution = value_text
            elif header_text == "Standing screen display size":
                screen_size = value_text
            elif header_text == "Hard Drive Size":
                memory = value_text
            elif header_text == "Hard Disk Description":
                memory += " " + value_text
            elif header_text == "Item Weight":
                weight = value_text
            elif header_text == "Operating System":
                opsys = value_text
            elif header_text == "Graphics Coprocessor":
                gpu = value_text
            elif header_text == "Processor Brand":
                cpu = value_text
            elif header_text == "Processor Type":
                cpu += " " + value_text

Saving amazon.htm to amazon (1).htm


In [20]:
print(cpu)

‎Intel ‎Celeron N4020


In [21]:
def clean_ram(ram_str):
    ram_cleaned = ''.join(filter(str.isdigit, ram_str))
    if ram_cleaned:
        return int(ram_cleaned)
    else:
        return 0

cleaned_ram = clean_ram(ram)
print(f"Original: {ram}, Cleaned: {cleaned_ram}")

Original: ‎8 GB, Cleaned: 8


In [22]:
# Function to clean and convert weight value
def convert_weight_format(weight_str):
    parts = weight_str.split()  # Split by space
    kg = float(parts[0].replace('kg', '').replace('\u200e', '').strip())  # Remove 'kg' and non-printing characters, then convert to float
    g = float(parts[1].replace('g', '').replace('\u200e', '').strip())  # Remove 'g' and non-printing characters, then convert to float
    total_weight = kg + (g / 1000)  # Convert grams to kilograms
    return round(total_weight, 3)  # Round to three decimal places

# Example weight value
weight = "‎1kg 680g"

# Apply the function to the weight value
cleaned_weight = convert_weight_format(weight)
print(f"Original: {weight}, Cleaned: {cleaned_weight} kg")


Original: ‎1kg 680g, Cleaned: 1.68 kg


In [23]:
# Function to extract and clean resolution values
def extract_resolution(resolution_str):
    parts = resolution_str.replace('pixel', '').replace('Pixel', '').replace('Pixels', '').strip().split('x')
    x_res = int(''.join(filter(str.isdigit, parts[0])))
    y_res = int(''.join(filter(str.isdigit, parts[1])))
    return x_res, y_res

# Example resolution value
resolution = "‎1920 x 1080 Pixels"

# Apply the function to the resolution value
x_res, y_res = extract_resolution(resolution)
print(f"Original: {resolution}, X_res: {x_res}, Y_res: {y_res}")


Original: ‎1920 x 1080 Pixels, X_res: 1920, Y_res: 1080


In [24]:
# Function to clean and convert screen size
def clean_screen_size(screen_size_str):
    # Remove any non-numeric characters except the decimal point and then convert to float
    screen_size = float(''.join(filter(lambda x: x.isdigit() or x == '.', screen_size_str)))
    return round(screen_size, 3)

# Example screen size value
screen_size = "‎14.0 Inches"

# Apply the function to the screen size value
cleaned_screen_size = clean_screen_size(screen_size)
print(f"Original: {screen_size}, Cleaned: {cleaned_screen_size}")

Original: ‎14.0 Inches, Cleaned: 14.0


In [25]:
import math
def calculate_ppi(x_res, y_res, screen_size):
    ppi = (math.sqrt(x_res**2 + y_res**2) / screen_size)
    return round(ppi, 3)

# Example values
x_res = 1920
y_res = 1080
screen_size = 14.0

# Calculate PPI
ppi = calculate_ppi(x_res, y_res, screen_size)
print(f"X_res: {x_res}, Y_res: {y_res}, Screen Size: {screen_size}, PPI: {ppi}")

X_res: 1920, Y_res: 1080, Screen Size: 14.0, PPI: 157.351


In [26]:
# Function to clean a single OS name
def clean_os(os_str):
    os_str = os_str.strip().lower()
    if 'windows' in os_str:
        return 'Windows'
    elif 'mac' in os_str or 'macos' in os_str:
        return 'MacOS'
    elif 'linux' in os_str:
        return 'Linux'
    else:
        return 'Other'

# Example OS value
os_value = "Windows 10 Home"

# Clean and standardize the OS value
cleaned_os_value = clean_os(os_value)
print(f"Original: {os_value}, Cleaned: {cleaned_os_value}")

Original: Windows 10 Home, Cleaned: Windows


In [27]:
# Check for Touchscreen and IPS in meta and title tags
meta_tag = soup.find('meta', attrs={'name': 'title'})
content_meta = meta_tag['content']
has_touchscreen_meta = 'touchscreen' in content_meta.lower()
has_ips_meta = 'ips' in content_meta.lower()

title_tag = soup.find('title')
content_title = title_tag.get_text()
has_touchscreen_title = 'touchscreen' in content_title.lower()
has_ips_title = 'ips' in content_title.lower()

has_touchscreen = 1 if (has_touchscreen_meta or has_touchscreen_title) else 0
has_ips = 1 if (has_ips_meta or has_ips_title) else 0

print(f"{has_touchscreen}")
print(f"{has_ips}")

0
0


In [28]:
hdd = 0

In [29]:
def fetch_processor(text):
    if text in ['Intel Core i7', 'Intel Core i5', 'Intel Core i3']:
        return text
    elif 'Intel' in text:
        return 'Other Intel Processor'
    else:
        return 'AMD Processor'

processed_processor = fetch_processor(cpu)
print(f"Original: {cpu}, Processed: {processed_processor}")

Original: ‎Intel ‎Celeron N4020, Processed: Other Intel Processor


In [35]:
import re

def extract_numeric_value(storage_string):
    # Use a regular expression to find the first sequence of digits in the string
    match = re.search(r'\d+', storage_string)
    if match:
        return float(match.group(0))
    else:
        return None

cleaned_memory = extract_numeric_value(memory)
print(f"Original: {memory}, Processed: {extract_numeric_value(memory)}")

Original: 256 GB SSD, Processed: 256.0


In [31]:
company = company.replace('\u200e', '')
gpu = gpu.replace('\u200e', '')
memory = memory.replace('\u200e', '')

In [36]:
data = [
    ['Company', 'Ram', 'Weight', 'Touchscreen', 'Ips', 'Ppi', 'Cpu brand', 'HDD', 'SSD', 'Gpu brand', 'os'],
    [company, cleaned_ram, cleaned_weight, has_touchscreen, has_ips, ppi, processed_processor, hdd, cleaned_memory, gpu, cleaned_os_value]
]
for row in data:
    print(row)

['Company', 'Ram', 'Weight', 'Touchscreen', 'Ips', 'Ppi', 'Cpu brand', 'HDD', 'SSD', 'Gpu brand', 'os']
['ASUS', 8, 1.68, 0, 0, 157.351, 'Other Intel Processor', 0, 256.0, 'Intel UHD Graphics', 'Windows']


In [37]:
headers = ['Company', 'Ram', 'Weight', 'Touchscreen', 'Ips', 'Ppi', 'Cpu brand', 'HDD', 'SSD', 'Gpu brand', 'os']
values = [company, cleaned_ram, cleaned_weight, has_touchscreen, has_ips, ppi, processed_processor, hdd, cleaned_memory, gpu, cleaned_os_value]

for header, value in zip(headers, values):
    print(f"{header}: {value}")

Company: ASUS
Ram: 8
Weight: 1.68
Touchscreen: 0
Ips: 0
Ppi: 157.351
Cpu brand: Other Intel Processor
HDD: 0
SSD: 256.0
Gpu brand: Intel UHD Graphics
os: Windows


In [39]:
import csv
from google.colab import files

# Define the output CSV file path
output_file_path = 'Product_Details.csv'

# Write to CSV file
with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data)

# Download the CSV file
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>